In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_accelerate-training-of-large-embedding-tables-by-lazyadam/nvidia_logo.png" style="width: 90px; float: right;">

# 1. Use multiple optimizers to accelerate training with LazyAdam

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

Merlin Models provide various model APIs for training, as shown in notebook [Iterating over Deep Learning Models using Merlin Models](https://nvidia-merlin.github.io/models/main/examples/03-Exploring-different-models.html). We can create a model, such as [Two Tower](https://nvidia-merlin.github.io/models/main/models_overview.html?highlight=two%20tower#two-tower), [DLRM](https://nvidia-merlin.github.io/models/main/examples/03-Exploring-different-models.html#dlrm-model) and so on, by simply one line: `model=mm.DLRMModel(schema)`. Some models contain large embedding tables, and training could be slow on such large sparse embeddings. However, this process could be accelerated by using a special optimizer, LazyAdam.

In this example, we utilize LazyAdam for large embedding tables and original Adam for other trainable weights to accelerate the whole training process.


**Learning objectives**
- Training a model with multiple optimizers
- Utilizing LazyAdam for training on large embedding tables

For this notebook, we use a single 32GB Quadro GV100 GPU and we report the training results and how much we speed up the model training time by using LazyAdam optimizer for large embedding tables below.

In [2]:
import os

import tensorflow as tf

import merlin.models.tf as mm
from merlin.datasets.synthetic import generate_data
from merlin.schema import Schema, Tags

2022-09-23 19:42:15.295442: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-23 19:42:16.444765: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:2d:00.0, compute capability: 7.0


## 1.1. Generate Synthetic Dataset
To generate the synthetic dataset for our example, we can use `generate_data()` function.

In [3]:
NUM_ROWS = 1000000
train, valid = generate_data("e-commerce-large", int(NUM_ROWS), set_sizes=(0.7, 0.3))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


Create a schema object and remove the target columns.

In [4]:
schema = train.schema.without(['click', 'conversion'])
train.schema = schema
valid.schema = schema

## 1.2. Build a Two-Tower model and train with Multiple Optimizers

In [5]:
item_embeddings = mm.Embeddings(schema.select_by_tag(Tags.ITEM), infer_embedding_sizes=True)
query_embeddings = mm.Embeddings(schema.select_by_tag(Tags.USER), infer_embedding_sizes=True)
model = mm.TwoTowerModel(schema, 
                         item_tower=mm.InputBlockV2(schema.select_by_tag(Tags.ITEM), categorical=item_embeddings).connect(mm.MLPBlock([64])), 
                         query_tower=mm.InputBlockV2(schema.select_by_tag(Tags.USER), categorical=query_embeddings).connect(mm.MLPBlock([64])),
                         samplers=[mm.InBatchSampler()],
)

The model initializer would infer the embedding table size from the schema, where the first dimension (`input_dim`) of each embedding table is the same as the cardinalities (categories) of each feature, and the second dimension is specified by the user. By setting `infer_embedding_sizes=True`, the initializer would infer the size based on the cardinalities: 
$$output\_dim=\left \lfloor cardinality^{0.25}\times multiplier \right \rfloor$$
The multiplier is 8 by default.To achieve the best performance with GPU operators, we adjust the embedding dimensions to multiples of 8.

### 1.2.1. Apply Multiple Optimizers to the Model

We usually set one optimizer to train a model, but for large embedding tables, at each batch, the weights to be updated could be really sparse, in other words, each time we only update the model based on a small batch of training data, so for a large embedding table (first dimension >>  batch size), at most batch_size rows would be updated. Thus in order to acceleate training on large embedding tables, we want to utilize the Lazy Adam for those large tables. 

Compared with Adam, Lazy Adam is optimized for sparse updates. It only update sparse variables indices for current batch. However it may result in slight difference in experiment results compared with Adam.

#### 1.2.1.1. Split Embedding Tables based on the First Dimension (`input_dim`)
Since we want to apply LazyAdam to the large tables, we have to split all tables into two sets. The result of `split_embeddings_on_size` (i.e. `item_large_tables` and `item_small_tables`) are two lists of embedding tables.


In [6]:
item_large_tables, item_small_tables = mm.split_embeddings_on_size(item_embeddings, threshold=1000)
query_large_tables, query_small_tables = mm.split_embeddings_on_size(query_embeddings, threshold=1000)

We can print out the size of each embedding table:

In [7]:
print("Large embedding tables of query tower:")
for t in query_large_tables:
    print(t.name, "first dimension: ", t.input_dim, "second dimension", t.dim)
print("Small embedding tables of query tower:")
for t in  query_small_tables:
    print(t.name, "first dimension: ", t.input_dim, "second dimension", t.dim)

Large embedding tables of query tower:
user_categories first dimension:  6087 second dimension 24
user_shops first dimension:  116742 second dimension 40
user_brands first dimension:  58016 second dimension 32
user_intentions first dimension:  33787 second dimension 32
user_id first dimension:  294737 second dimension 48
Small embedding tables of query tower:
user_profile first dimension:  99 second dimension 8
user_group first dimension:  15 second dimension 8
user_gender first dimension:  4 second dimension 8
user_age first dimension:  9 second dimension 8
user_consumption_1 first dimension:  5 second dimension 8
user_consumption_2 first dimension:  5 second dimension 8
user_is_occupied first dimension:  4 second dimension 8
user_geography first dimension:  6 second dimension 8


#### 1.2.1.2. Set MultiOptimizer

The `MultiOptimizer` module enables multiple optimizers [(e.g. Adam, SGD, RMSProp, Adagrad)](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) to be used in different layers in parallel. Here we want to apply `LazyAdam` for large embedding tables, and `Adam` for the small embedding tables and all other layers.

In [8]:
optimizer = mm.MultiOptimizer(
                default_optimizer="adam",
                optimizers_and_blocks=[mm.OptimizerBlocks(mm.LazyAdam(), item_large_tables + query_large_tables),
                                       mm.OptimizerBlocks("adam", item_small_tables + query_small_tables)]
                )

### 1.2.2. Train the Model

In [9]:
model.compile(optimizer=optimizer)
model.fit(train, batch_size=1024, epochs=1)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The sampler InBatchSampler returned no samples for this batch.


684/684 [==============================] - 30s 27ms/step - loss: 6.9128 - recall_at_10: 0.0278 - mrr_at_10: 0.0236 - ndcg_at_10: 0.0246 - map_at_10: 0.0236 - precision_at_10: 0.0028 - regularization_loss: 0.0000e+00


Note all other trainable parameters not specified an optimizer would use the `default_optimizer`.

## 1.3. Build a Two-Tower model and train with a single optimizer

Now, let's create the same model but this time compile it only with `Adam` optimizer. 

In [10]:
item_embeddings = mm.Embeddings(schema.select_by_tag(Tags.ITEM), infer_embedding_sizes=True)
query_embeddings = mm.Embeddings(schema.select_by_tag(Tags.USER), infer_embedding_sizes=True)
model = mm.TwoTowerModel(schema, 
                         item_tower=mm.InputBlockV2(schema.select_by_tag(Tags.ITEM), categorical=item_embeddings).connect(mm.MLPBlock([64])), 
                         query_tower=mm.InputBlockV2(schema.select_by_tag(Tags.USER), categorical=query_embeddings).connect(mm.MLPBlock([64])),
                         samplers=[mm.InBatchSampler()],
)

### 1.3.1. Train the Model

In [11]:
model.compile(optimizer="adam")
model.fit(train, batch_size=1024, epochs=1)

The sampler InBatchSampler returned no samples for this batch.


684/684 [==============================] - 72s 100ms/step - loss: 6.9128 - recall_at_10: 0.0280 - mrr_at_10: 0.0225 - ndcg_at_10: 0.0238 - map_at_10: 0.0225 - precision_at_10: 0.0028 - regularization_loss: 0.0000e+00


## 1.4. Compare model training times with multiple optimizers vs single optimizers

We created the same model but this time compile it with `Adam` optimizer. The training result shows that for each step, it costs about `72s`. And as shown in the experiment above, the training time with `LazyAdam` optimizer is about `30s`, it achieves about `2.4X` speed up without any degradation in the mrr and ndcg metric values.

That's it! In this example, you learned how to use multiple optimizers for training a Two-Tower model, where one of the optimizers is `LazyAdam` which is variant of the Adam optimizer that handles sparse updates more.